# 2: Extract Local Weighting:

In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import lime
import lime.lime_tabular
import deeplift

from collections import Counter

from copy import deepcopy

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation
from keras import backend as K

# For Deep Learning Explanations
from deepexplain.tensorflow import DeepExplain
from deeplift.conversion import kerasapi_conversion as kc

Using TensorFlow backend.


In [4]:
# Load Data
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

oh_y_train = np.load("oh_y_train.npy")
oh_y_test = np.load("oh_y_test.npy")

In [5]:
#### NB: MAKE SURE CORRECT MODEL
model = load_model("NN.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.




In [6]:
# For indexing
nn_preds_test = model.predict_classes(X_test)
nn_preds_train = model.predict_classes(X_train)

## Get Activations:

In [7]:
def get_activations(model, model_inputs, print_shape_only=False, layer_name=None):

    activations = []
    inp = model.input

    model_multi_inputs_cond = True
    if not isinstance(inp, list):
        # only one input! let's wrap it in a list.
        inp = [inp]
        model_multi_inputs_cond = False

    outputs = [layer.output for layer in model.layers if
               layer.name == layer_name or layer_name is None]  # all layer outputs

    funcs = [K.function(inp + [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

    if model_multi_inputs_cond:
        list_inputs = []
        list_inputs.extend(model_inputs)
        list_inputs.append(0.)
    else:
        list_inputs = [model_inputs, 0.]

    # Learning phase. 0 = Test mode (no dropout or batch normalization)
    # layer_outputs = [func([model_inputs, 0.])[0] for func in funcs]
    layer_outputs = [func(list_inputs)[0] for func in funcs]
    for layer_activations in layer_outputs:
        activations.append(layer_activations)

    return activations


X_train_act = list()
X_test_act = list()

for i in range(0, 60000, 1000):
    print(i)
    start = i
    end = i + 1000
    
    X_train_act_seg = get_activations(model, X_train[start: end])[-8]
    X_train_act.append(X_train_act_seg)
    
    
for i in range(0, 10000, 1000):
    print(i)
    start = i
    end = i + 1000
    
    X_test_act_seg = get_activations(model, X_test[start: end])[-8]
    X_test_act.append(X_test_act_seg)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [8]:
X_train_act_new = list()
X_test_act_new = list()

for group in X_train_act:
    for i in range(len(group)):
        X_train_act_new.append(group[i])
                
for group in X_test_act:
    for i in range(len(group)):
        X_test_act_new.append(group[i])        

X_train_act = np.array(X_train_act_new)
X_test_act = np.array(X_test_act_new)

np.save("X_train_act", X_train_act)
np.save("X_test_act", X_test_act)

## Get DeepLIFT Contributions:
https://github.com/kundajelab/deeplift

In [9]:
from copy import deepcopy

deeplift_model =\
    kc.convert_model_from_saved_files(
        "NN.h5",
        nonlinear_mxts_mode=deeplift.layers.NonlinearMxtsMode.DeepLIFT_GenomicsDefault) 
    
# Which layer to propagate contribution scores?
find_scores_layer_idx = -8

deeplift_contribs_func = deeplift_model.get_target_contribs_func(
                            find_scores_layer_idx=find_scores_layer_idx,
                            target_layer_idx=-2)


# Need to iterate scores for each output neuron one at a time (10 for MNIST)
for i in range(model.get_weights()[-1].shape[0]):
    
    train = np.array(deeplift_contribs_func(task_idx=i,
                                             input_data_list=[X_train],
                                             batch_size=10,
                                             progress_update=1000))

    test = np.array(deeplift_contribs_func(task_idx=i,
                                             input_data_list=[X_test],
                                             batch_size=10,
                                             progress_update=1000))
    
    train = np.array(train)
    test = np.array(test)
    train = np.expand_dims(train, 1)
    test = np.expand_dims(test, 1)
    
    if i == 0:
        X_train_deeplift = deepcopy(train)
        X_test_deeplift = deepcopy(test)
    else:
        X_train_deeplift = np.append(X_train_deeplift, train, axis=1)
        X_test_deeplift = np.append(X_test_deeplift, test, axis=1)

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault



/Users/eoinkenny/Desktop/IJCAI-2019-master/myenv/lib/python3.7/site-packages/deeplift/conversion/kerasapi_conversion.py:359: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  str_data = h5py.File(h5_file).attrs["model_config"]
/Users/eoinkenny/Desktop/IJCAI-2019-master/myenv/lib/python3.7/site-packages/deeplift/conversion/kerasapi_conversion.py:366: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for det

For layer 1 the preceding linear layer is 0 of type Conv2D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
For layer 4 the preceding linear layer is 3 of type Conv2D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
For layer 8 the preceding linear layer is 7 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
For layer 11 the preceding linear layer is 10 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
Heads-up: I assume softmax is the output layer, not an intermediate one; if it

Done 49000
Done 50000
Done 51000
Done 52000
Done 53000
Done 54000
Done 55000
Done 56000
Done 57000
Done 58000
Done 59000
No reference provided - using zeros
Done 0
Done 1000
Done 2000
Done 3000
Done 4000
Done 5000
Done 6000
Done 7000
Done 8000
Done 9000
No reference provided - using zeros
Done 0
Done 1000
Done 2000
Done 3000
Done 4000
Done 5000
Done 6000
Done 7000
Done 8000
Done 9000
Done 10000
Done 11000
Done 12000
Done 13000
Done 14000
Done 15000
Done 16000
Done 17000
Done 18000
Done 19000
Done 20000
Done 21000
Done 22000
Done 23000
Done 24000
Done 25000
Done 26000
Done 27000
Done 28000
Done 29000
Done 30000
Done 31000
Done 32000
Done 33000
Done 34000
Done 35000
Done 36000
Done 37000
Done 38000
Done 39000
Done 40000
Done 41000
Done 42000
Done 43000
Done 44000
Done 45000
Done 46000
Done 47000
Done 48000
Done 49000
Done 50000
Done 51000
Done 52000
Done 53000
Done 54000
Done 55000
Done 56000
Done 57000
Done 58000
Done 59000
No reference provided - using zeros
Done 0
Done 1000
Done 2000


In [10]:
train = deepcopy(X_train_deeplift)
test = deepcopy(X_test_deeplift)

In [11]:
X_train_deeplift = list()
X_test_deeplift = list()

for i in range(len(nn_preds_train)):
    index = nn_preds_train[i]
    X_train_deeplift.append(train[i][index])

for i in range(len(nn_preds_test)):
    index = nn_preds_test[i]
    X_test_deeplift.append(test[i][index])

In [12]:
X_train_deeplift = np.array(X_train_deeplift)
X_test_deeplift = np.array(X_test_deeplift)

In [13]:
print("Training:", X_train_deeplift.shape)
print("Testing:", X_test_deeplift.shape)

np.save("new_X_train_deeplift", X_train_deeplift)
np.save("new_X_test_deeplift", X_test_deeplift)

Training: (60000, 128)
Testing: (10000, 128)
